In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 448 kB 45.2 MB/s 
     |████████████████████████████████| 86 kB 5.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [44]:
import pandas as pd
from collections import defaultdict
from konlpy.tag import Hannanum, Kkma, Komoran, Okt
import csv

url = 'https://raw.githubusercontent.com/JunHCha/K-POP-Lyrics-1964-2020/main/data/melon_lyric_preprocess.csv'
df = pd.read_csv(url)


In [45]:
okt = Okt()

word_cnt_df = pd.DataFrame(columns=['genre', 'year', 'word', 'freq'])

genres = df['genre'].unique().tolist()
print(f'total {len(genres)} genres')

for genre in genres[31:]:
    genre_df = df.loc[df['genre'] == genre]
    years = genre_df['year'].unique().tolist()

    for year in years:
        genre_year_df = genre_df.loc[genre_df['year'] == year]
        genre_freq_df = pd.DataFrame(columns=['genre', 'year', 'word', 'freq'])

        for index, row in genre_year_df.iterrows():
            freq = defaultdict(int)
            lyric = row['lyric']
            words = [word[0] for word in okt.pos(lyric) if word[1] == 'Noun' or word[1] == 'Adjective']

            for token in words:
                token = token.strip()
                freq[token] += 1

            current = pd.DataFrame(
                {
                    "genre": genre,
                    "year" : year,
                    "word" : [word for word in freq],
                    "freq" : [freq[word] for word in freq],
                }
            )

            genre_freq_df = pd.concat([genre_freq_df, current])
        
        genre_freq_df['freq'] = genre_freq_df.groupby(['genre', 'year',  'word'])['freq'].transform('sum')
        genre_freq_df = genre_freq_df.drop_duplicates(subset=['genre', 'year', 'word'])
        genre_freq_df = genre_freq_df.loc[genre_freq_df['freq'] >= 10]
    
        word_cnt_df = pd.concat([word_cnt_df, genre_freq_df])


total 43 genres


In [46]:
word_cnt_df.to_csv('lyric_freq_genre_31-42.csv')
from google.colab import files
files.download('lyric_freq_genre_31-42.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>